In [ ]:
import mne
import time
import os
import numpy as np

In [ ]:
data_path = 'D:/Diplomarbeit_data/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']

mne.set_log_level('WARNING')

In [ ]:
# Load eye data:
src_path = data_path + 'bad_ch_interpolated'
# There can be only one file  with matching conditions since we are splitting in folders:
f_name = [f for f in os.listdir(src_path) if ('A01' in f) and ('eye' in f)][0]

file = src_path + '/' + f_name
raw = mne.io.read_raw(file, preload=True)

raw.info

In [ ]:
# Extract the EOG data:
eog_r = raw.get_data(picks='EOGL')
eog_l = raw.get_data(picks='EOGR')
eog_c = raw.get_data(picks='EOGC')

# Calculate EOG derivatives:
heog = eog_r - eog_l
veog = eog_c - (eog_r + eog_l) / 2
reog = (eog_c + eog_r + eog_l) / 3

# Create nchannels x ntimes matrix:
eog_derivatives = np.vstack((heog, veog, reog))

# Create an info file to later add the channels:
info = mne.create_info(['EOGH', 'EOGV', 'EOGRad'], 200.0, ch_types='eog')

# Create a raw array for the eog derivatives:
raw_derivatives = mne.io.RawArray(eog_derivatives, info, first_samp=raw.first_samp)

raw = raw.add_channels([raw_derivatives], force_update_info=True)

In [ ]:
raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False, title=f'EOG derivatives added.')

In [ ]:
# Lowpass filter the eog derivatives:
raw = raw.copy().filter(l_freq=None, h_freq=5.0, picks=['eog'], method='iir')

In [ ]:
raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False, title=f'EOG lowpass filtered.')

In [ ]:
def get_subset_of_dict(full_dict, keys_of_interest):
    return dict((k, full_dict[k]) for k in keys_of_interest if k in full_dict)

In [ ]:
# Extract epochs:
events_from_annot, event_dict = mne.events_from_annotations(raw)

#
markers_of_interest = ['Blink', 'Rest' ,'Horz', 'Vert']

event_dict_of_interest = get_subset_of_dict(event_dict, markers_of_interest)

# TODO select event ID's of interest, hand over dict for event_id to make it easier to extract them:
epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict_of_interest, tmin=2.0, tmax=10.0, baseline=None, reject_by_annotation=True, preload=True, picks=['eeg', 'eog'])


In [ ]:
epochs['Rest'].plot()